# Build Reference Standard Masks

In this notebook there is all the code needed to build the binary and gaussian reference standard masks used for evaluation. 

Requirements not available in this repository:
* To be able to run this code one needs to run it in an eviroment with the python packages: __numpy__, __SimpleITK__.
* To be able to run this code one needs the __MRI sequences__ for each subject wich is not uploaded.
* To be able to run this code one needs to run it in an __Ubuntu__ distribution with __sudo__ privilegies as well as the __fsl__ package installed on the root and with the __FSLPATH__ set in the bash profile. 

In [1]:
import numpy as np
import SimpleITK as sitk
import os
import subprocess

from utils import excel_data, get_subject_path

In [2]:
subjects_data = {}

# Join subjects data for easier use
for row in excel_data:
    code = row['CODI'][:15]  # Subject identifier code
    data = {
            'size': row['SIZE'],
            'coordinates': np.array([row['X'], row['Y'], row['Z']]),
        }
    if code in subjects_data.keys():
        subjects_data[code].append(data)
    else:
        subjects_data[code] = [data]

In [3]:
images_shape = np.array([22, 512, 512])  # Y Z X
gaussian_filter_mm = 2
mask_name = 'wmh.nii.gz'
gaussian_mask_name = 'wmh_gauss.nii.gz'

# Loop over all subjects
for subject_code, subject_data in list(subjects_data.items()):
    
    # Create mask arrays
    test_array = np.zeros(images_shape, dtype=np.float)
    for element in subject_data:
        x, y, z = element['coordinates'].astype(int) - 1  # Move origin to 0.
        test_array[y, z, x] += 1
    test_array = np.flip(test_array, (0, 2))  # Axis Y and X are inverted when visualizing
    
    # Check that exists the data for this subject
    subject_path = get_subject_path(subject_code)
    if not os.path.isdir(subject_path):
        print(subject_path)
        continue
    
    # Create image and add metadata from original FLAIR space
    test_image = sitk.GetImageFromArray(test_array)
    flair_image = sitk.ReadImage(f'{subject_path}input/orig/FLAIR.nii.gz')
    test_image.CopyInformation(flair_image)
    
    # Write mask
    mask_path = f'{subject_path}input/{mask_name}'
    sitk.WriteImage(test_image, mask_path)
    
    # Apply FSL gaussian filter
    bash_command = f'cd {subject_path}/input; fslmaths {mask_name} -s {gaussian_filter_mm} {gaussian_mask_name}'
    if not subprocess.run(bash_command, shell=True):
        raise Exception('Something went wrong while applying the gaussian filter.')
